In [1]:
# Load dataset

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

'''
for dirname, _, filenames in os.walk('/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''

# Define data transformations (e.g., flipping and normalization)
transform = transforms.Compose([
    transforms.ToTensor(), # Convert images to PyTorch tensors
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values to [-1, 1]
])

# Create train and test set from directories
'''
train_set = datasets.ImageFolder(root='/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training', transform=transform)
test_set = datasets.ImageFolder(root='/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test', transform=transform)
'''
dataset = ImageFolder(root='/kaggle/input/reduced-dataset/dataset', transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create data loaders for training and test sets
batch_size = 32  # You can adjust this according to your system's memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [2]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Model - used pre-trained ones from pytorch
model = models.resnet50()

'''
# Freeze all layers except the final classification layer
for param in model.parameters():
    param.requires_grad = False
'''

num_classes = 6
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Replace the final fully connected layer

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.01, momentum=0.9)

In [3]:
# Train
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 100  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    totalTrainLoss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        (inputs, labels) = (inputs.to(device), labels.to(torch.long).to(device))
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        totalTrainLoss += loss
    print("Epoch", epoch, "Training Loss:", totalTrainLoss.item())

Epoch 0 Training Loss: 1790.3992919921875
Epoch 1 Training Loss: 983.5216064453125
Epoch 2 Training Loss: 717.9075927734375
Epoch 3 Training Loss: 584.914306640625
Epoch 4 Training Loss: 562.4871826171875
Epoch 5 Training Loss: 553.5817260742188
Epoch 6 Training Loss: 472.6867370605469
Epoch 7 Training Loss: 560.9715576171875
Epoch 8 Training Loss: 488.4855041503906
Epoch 9 Training Loss: 501.5417175292969
Epoch 10 Training Loss: 534.8219604492188
Epoch 11 Training Loss: 510.9921569824219
Epoch 12 Training Loss: 521.037353515625
Epoch 13 Training Loss: 502.8099060058594
Epoch 14 Training Loss: 482.532958984375
Epoch 15 Training Loss: 452.2392883300781
Epoch 16 Training Loss: 414.15863037109375
Epoch 17 Training Loss: 518.8253173828125
Epoch 18 Training Loss: 489.9633483886719
Epoch 19 Training Loss: 565.2818603515625
Epoch 20 Training Loss: 471.59796142578125
Epoch 21 Training Loss: 433.0090637207031
Epoch 22 Training Loss: 404.0568542480469
Epoch 23 Training Loss: 431.2936096191406
Ep

In [2]:
# Test
model.eval()
corrects = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        corrects += torch.sum(preds == labels.data)

test_acc = corrects.double() / len(test_dataset)
print(f'Test Accuracy: {test_acc:.4f}')

# Specify the file path where you want to save the model
save_path = '/kaggle/working/model.pth'

# Save the model (including architecture and weights)
torch.save(model, save_path)


Test Accuracy: 0.8021


In [7]:
#second test
from PIL import Image

# map class names to class number
root_directory = '/kaggle/input/reduced-dataset/dataset'
class_directories = sorted([d for d in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory, d))])
class_mapping = {}
for i, class_dir in enumerate(class_directories):
    class_mapping[i] = class_dir
    
#define GPU usage
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the entire model
model = torch.load(save_path)

# Set the model to evaluation mode
model.eval()

# Load and preprocess the new image
transform = transforms.Compose([
    transforms.Resize((100, 100)),  # Resize as per your model's input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values to [-1, 1]
])

image_path = '/kaggle/input/irl-test/bunch of banana.jpg'
image = Image.open(image_path)
image = transform(image).unsqueeze(0)  # Add a batch dimension
image = image.to(device) # convert data to be GPU suitable

# Perform inference
with torch.no_grad():
    outputs = model(image)

# Convert probabilities to class scores
class_scores = torch.softmax(outputs, dim=1)

# Get the predicted class
_, predicted_class = torch.max(class_scores, 1)

# Print or use the predicted class and class scores
print(f'Predicted Class: {predicted_class.item()}')
print(f'Predicted Class: {class_mapping[predicted_class.item()]}')

Predicted Class: 1
Predicted Class: Banana
